# Presentation

1. Rider demand by month, compare years
2. Rider demand by day of week, compare years
3. Rider demand by hour, compare years
4. Rider demand by hour for each day of week
5. See which stations are in the most demand for the highest demand hours on different days
1. This probably can't be on one graph, probably by day

## Aggregate Temp Tables

In [61]:
-- temp tables for aggregates
create temporary table agg_2018 as

select
To_Char(start_time, 'YYYY') as year,
To_Char(start_time, 'MM') as month_no,
To_Char(start_time, 'ID') as day_no,
To_Char(start_time, 'HH24') as hour_no,
To_Char(start_time, 'Month') as month,
To_Char(start_time, 'Day') as day,
To_Char(start_time, 'HH12') || ':00 ' || To_Char(start_time, 'am') as hour,
count(1) as riders

from bluebikes_2018
group by year, month_no, day_no, hour_no, month, day, hour
order by month_no, day_no, hour_no


Commands completed successfully

Total execution time: 00:00:04.287

In [63]:
-- temp tables for aggregates
create temporary table agg_2019 as

select
To_Char(start_time, 'YYYY') as year,
To_Char(start_time, 'MM') as month_no,
To_Char(start_time, 'ID') as day_no,
To_Char(start_time, 'HH24') as hour_no,
To_Char(start_time, 'Month') as month,
To_Char(start_time, 'Day') as day,
To_Char(start_time, 'HH12') || ':00 ' || To_Char(start_time, 'am') as hour,
count(1) as riders

from bluebikes_2019
group by year, month_no, day_no, hour_no, month, day, hour
order by month_no, day_no, hour_no


Commands completed successfully

Total execution time: 00:00:06.260

In [65]:
select * from agg_2018
limit 5;

select * from agg_2019
limit 5;

(5 row(s) affected)

(5 row(s) affected)

Total execution time: 00:00:00.403

year,month_no,day_no,hour_no,month,day,hour,riders
2018,01,1,00,January,Monday,12:00 am,47
2018,01,1,01,January,Monday,01:00 am,19
2018,01,1,02,January,Monday,02:00 am,13
2018,01,1,03,January,Monday,03:00 am,3
2018,01,1,04,January,Monday,04:00 am,7


year,month_no,day_no,hour_no,month,day,hour,riders
2019,01,1,00,January,Monday,12:00 am,24
2019,01,1,01,January,Monday,01:00 am,16
2019,01,1,02,January,Monday,02:00 am,11
2019,01,1,03,January,Monday,03:00 am,3
2019,01,1,04,January,Monday,04:00 am,15


## 1\. Rider demand by month, compare years

In [114]:
-- select 
-- a.month,
-- sum(a.riders) as riders_2018
-- from agg_2018 a
-- group by a.month, a.month_no
-- order by a.month_no;

-- select 
-- a.month,
-- sum(a.riders) as riders_2019
-- from agg_2019 a
-- group by a.month, a.month_no
-- order by a.month_no;

select 
a.month,
sum(a.riders) as riders_2018,
sum(b.riders) as riders_2019,
sum(b.riders) - sum(a.riders) as riders_change,
round((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders), 4) * 100 || ' %' as percent_change
from agg_2018 a
join agg_2019 b on a.month_no = b.month_no and a.day_no = b.day_no and a.hour_no = b.hour_no
group by a.month, a.month_no
order by a.month_no;

(12 row(s) affected)

Total execution time: 00:00:00.202

month,riders_2018,riders_2019,riders_change,percent_change
January,40932,69872,28940,70.7000 %
February,62817,80466,17649,28.1000 %
March,62985,102369,39384,62.5300 %
April,98194,166694,68500,69.7600 %
May,178865,223084,44219,24.7200 %
June,205359,274022,68663,33.4400 %
July,242916,316931,74015,30.4700 %
August,236076,337443,101367,42.9400 %
September,236182,363185,127003,53.7700 %
October,200100,305504,105404,52.6800 %


# Rider Demand by Day of Week, Compare Years

In [148]:
select 
a.day,
sum(a.riders) as riders_2018,
sum(b.riders) as riders_2019,
sum(b.riders) - sum(a.riders) as riders_change,
-- min(sum(b.riders) - sum(a.riders)) over() as min_riders_change,
-- max(sum(b.riders) - sum(a.riders)) over() as max_riders_change,
round(((sum(b.riders) - sum(a.riders)) - (min(sum(b.riders) - sum(a.riders)) over()))    /    ((max(sum(b.riders) - sum(a.riders)) over()) - (min(sum(b.riders) - sum(a.riders)) over())), 4) as relative_rider_change,

round((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders), 4) * 100 || ' %' as percent_change,
-- max((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over() as max_change,
-- min((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over() as min_change,
-- ((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders) - (min((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over())) as new_value, -- this is good
round(((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders) - (min((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over())) / ((max((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over()) - (min((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over())), 4) * 100 || ' %' as min_max_scaled_change,
-- ((percent change of riders) - (minimum of percent change of riders)) / ((maximum of percent change of riders) - (minimum of percent change of riders))
round(((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) / avg((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over() - 1, 4) * 100 || ' %' as normalized_abolute_change

from agg_2018 a
join agg_2019 b on a.month_no = b.month_no and a.day_no = b.day_no and a.hour_no = b.hour_no
group by a.day, a.day_no
order by a.day_no;

(7 row(s) affected)

Total execution time: 00:00:00.178

day,riders_2018,riders_2019,riders_change,relative_rider_change,percent_change,min_max_scaled_change,normalized_abolute_change
Monday,256032,372387,116355,0.8831,45.4500 %,57.2000 %,4.5500 %
Tuesday,265161,384455,119294,1.0000,44.9900 %,54.7000 %,3.5000 %
Wednesday,288485,389421,100936,0.2699,34.9900 %,0.0000 %,-19.5000 %
Thursday,282700,386579,103879,0.3870,36.7500 %,9.6100 %,-15.4600 %
Friday,276229,385468,109239,0.6001,39.5500 %,24.9300 %,-9.0200 %
Saturday,208144,319024,110880,0.6654,53.2700 %,100.0000 %,22.5600 %
Sunday,191055,285203,94148,0.0000,49.2800 %,78.1600 %,13.3700 %


# Rider Demand by Hour of Day, Compare Years

In [150]:
select 
a.hour,
sum(a.riders) as riders_2018,
sum(b.riders) as riders_2019,
sum(b.riders) - sum(a.riders) as riders_change,
round((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders), 4) * 100 || ' %' as percent_change,
round(((sum(b.riders) - sum(a.riders)) - (min(sum(b.riders) - sum(a.riders)) over()))    /    ((max(sum(b.riders) - sum(a.riders)) over()) - (min(sum(b.riders) - sum(a.riders)) over())), 4) as relative_rider_change,
round(((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders) - (min((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over())) / ((max((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over()) - (min((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over())), 4) * 100 || ' %' as min_max_scaled_change,
round(((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) / avg((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over() - 1, 4) * 100 || ' %' as normalized_abolute_change
from agg_2018 a
join agg_2019 b on a.month_no = b.month_no and a.day_no = b.day_no and a.hour_no = b.hour_no
group by a.hour, a.hour_no
order by a.hour_no;

(24 row(s) affected)

Total execution time: 00:00:00.190

hour,riders_2018,riders_2019,riders_change,percent_change,relative_rider_change,min_max_scaled_change,normalized_abolute_change
12:00 am,14381,20420,6039,41.9900 %,0.0629,41.2700 %,-3.0800 %
01:00 am,9333,14251,4918,52.6900 %,0.0493,100.0000 %,21.6200 %
02:00 am,5968,8752,2784,46.6500 %,0.0234,66.8200 %,7.6700 %
03:00 am,2217,3078,861,38.8400 %,0.0000,23.9400 %,-10.3600 %
04:00 am,2339,3394,1055,45.1000 %,0.0024,58.3400 %,4.1000 %
05:00 am,10571,15065,4494,42.5100 %,0.0441,44.1200 %,-1.8800 %
06:00 am,35046,47183,12137,34.6300 %,0.1370,0.8700 %,-20.0700 %
07:00 am,96792,130160,33368,34.4700 %,0.3950,0.0000 %,-20.4300 %
08:00 am,167768,229859,62091,37.0100 %,0.7441,13.9200 %,-14.5800 %
09:00 am,98617,141669,43052,43.6600 %,0.5127,50.3900 %,0.7600 %


# Rider Demand by Hour of Every Day, Compare Years

In [151]:
select
a.day,
a.hour,
sum(a.riders) as riders_2018,
sum(b.riders) as riders_2019,
sum(b.riders) - sum(a.riders) as riders_change,
round((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders), 4) * 100 || ' %' as percent_change,
round(((sum(b.riders) - sum(a.riders)) - (min(sum(b.riders) - sum(a.riders)) over()))    /    ((max(sum(b.riders) - sum(a.riders)) over()) - (min(sum(b.riders) - sum(a.riders)) over())), 4) as relative_rider_change,
round(((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders) - (min((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over())) / ((max((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over()) - (min((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over())), 4) * 100 || ' %' as min_max_scaled_change,
round(((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) / avg((sum(b.riders) - sum(a.riders)::numeric) / sum(a.riders)) over() - 1, 4) * 100 || ' %' as normalized_abolute_change
from agg_2018 a
join agg_2019 b on a.month_no = b.month_no and a.day_no = b.day_no and a.hour_no = b.hour_no
group by a.day, a.day_no, a.hour, a.hour_no
order by a.day_no, a.hour_no;

(168 row(s) affected)

Total execution time: 00:00:00.226

day,hour,riders_2018,riders_2019,riders_change,percent_change,relative_rider_change,min_max_scaled_change,normalized_abolute_change
Monday,12:00 am,1574,2139,565,35.9000 %,0.0313,27.4100 %,-18.7900 %
Monday,01:00 am,882,1223,341,38.6600 %,0.0182,30.7100 %,-12.5400 %
Monday,02:00 am,446,707,261,58.5200 %,0.0136,54.4000 %,32.3900 %
Monday,03:00 am,224,289,65,29.0200 %,0.0022,19.2000 %,-34.3500 %
Monday,04:00 am,347,552,205,59.0800 %,0.0103,55.0700 %,33.6500 %
Monday,05:00 am,1860,2580,720,38.7100 %,0.0403,30.7600 %,-12.4300 %
Monday,06:00 am,6018,8431,2413,40.1000 %,0.1389,32.4200 %,-9.2900 %
Monday,07:00 am,16947,23693,6746,39.8100 %,0.3912,32.0700 %,-9.9500 %
Monday,08:00 am,29106,40835,11729,40.3000 %,0.6814,32.6600 %,-8.8400 %
Monday,09:00 am,14872,22167,7295,49.0500 %,0.4232,43.1100 %,10.9700 %


In [131]:
-- THIS IS NOT WORKING
select
a.day,
a.hour,
avg(avg(a.riders)) over(partition by a.hour_no) as calc_riders_2018,
avg(a.riders) as riders_2018,
avg(b.riders) as riders_2019,
avg(b.riders) - avg(a.riders) as riders_change,
round((avg(b.riders) - avg(a.riders)::numeric) / avg(a.riders), 4) * 100 || ' %' as percent_change
from agg_2018 a
join agg_2019 b on a.month_no = b.month_no and a.day_no = b.day_no and a.hour_no = b.hour_no
group by a.day, a.day_no, a.hour, a.hour_no
order by a.day_no, a.hour_no;

(168 row(s) affected)

Total execution time: 00:00:00.252

day,hour,calc_riders_2018,riders_2018,riders_2019,riders_change,percent_change
Monday,12:00 am,171.2023809523809524,131.1666666666666667,178.2500000000000000,47.0833333333333333,35.9000 %
Monday,01:00 am,111.1071428571428572,73.5000000000000000,101.9166666666666667,28.4166666666666667,38.6600 %
Monday,02:00 am,71.0476190476190476,37.1666666666666667,58.9166666666666667,21.7500000000000000,58.5200 %
Monday,03:00 am,26.3928571428571428,18.6666666666666667,24.0833333333333333,5.4166666666666666,29.0200 %
Monday,04:00 am,27.8452380952380952,28.9166666666666667,46.0000000000000000,17.0833333333333333,59.0800 %
Monday,05:00 am,125.8452380952380952,155.0000000000000000,215.0000000000000000,60.0000000000000000,38.7100 %
Monday,06:00 am,417.2142857142857143,501.5000000000000000,702.5833333333333333,201.0833333333333333,40.1000 %
Monday,07:00 am,1152.2857142857142857,1412.2500000000000000,1974.4166666666666667,562.1666666666666667,39.8100 %
Monday,08:00 am,1997.2380952380952381,2425.5000000000000000,3402.9166666666666667,977.4166666666666667,40.3000 %
Monday,09:00 am,1174.0119047619047619,1239.3333333333333333,1847.2500000000000000,607.9166666666666667,49.0500 %


# Rider By Type

In [14]:
create TEMPORARY table bluebikes_combined as

select '2018' as data_year, bike_id, start_time, end_time, start_station_id, end_station_id, user_type, user_birth_year, user_gender
from bluebikes_2018
union all
select '2019' as data_year, bike_id, start_time, end_time, start_station_id, end_station_id, user_type, user_birth_year, user_gender
from bluebikes_2019


Commands completed successfully

Total execution time: 00:00:02.598

In [15]:
select * from bluebikes_combined
order by random()
limit 100

(100 row(s) affected)

Total execution time: 00:00:02.052

data_year,bike_id,start_time,end_time,start_station_id,end_station_id,user_type,user_birth_year,user_gender
2018,2968,2018-05-15T09:27:50.938000,2018-05-15T18:20:04.205000,67,78,Subscriber,1990,2
2019,4116,2019-07-03T07:16:40.081000,2019-07-03T07:21:01.678000,105,87,Subscriber,1988,1
2019,2152,2019-04-17T20:36:25.167000,2019-04-17T20:41:19.657000,67,68,Subscriber,1996,1
2019,2884,2019-07-19T23:00:52.123000,2019-07-19T23:12:58.209000,335,179,Subscriber,1995,1
2018,3474,2018-10-23T18:28:39.509000,2018-10-23T18:43:27.280000,335,149,Subscriber,1989,1
2018,2591,2018-08-23T14:37:18.195000,2018-08-23T14:56:54.425000,40,72,Customer,1969,0
2019,3688,2019-04-13T16:59:10.104000,2019-04-13T17:01:10.407000,67,67,Subscriber,1997,2
2019,4862,2019-08-24T23:19:13.413000,2019-08-24T23:35:12.952000,49,342,Subscriber,1983,1
2019,4443,2019-08-27T17:27:27.613000,2019-08-27T17:33:09.383000,115,71,Subscriber,1966,1
2019,3871,2019-08-30T07:02:55.038000,2019-08-30T07:09:22.838000,331,55,Customer,1997,1


In [2]:
-- Rider type by category overall

select user_type,
count(*) / (cast(sum(count(*)) over() as float)) as percentage
from bluebikes_2018
group by user_type;


select user_type,
count(*) / (cast(sum(count(*)) over() as float)) as percentage
from bluebikes_2019
group by user_type;

(2 row(s) affected)

(2 row(s) affected)

Total execution time: 00:00:01.358

user_type,percentage
Customer,0.18731071169574037
Subscriber,0.8126892883042597


user_type,percentage
Customer,0.21171939202477505
Subscriber,0.7882806079752249


In [3]:
-- Rider type by category by quarter

with q as(
    select user_type,
    ntile(4) over(order by start_time) as quarter
    from bluebikes_2018
    )
select user_type, quarter,
count(*) as rides,
count(*) / (cast(sum(count(*)) over(partition by quarter) as float)) as percentage
from q
group by quarter, user_type
order by quarter, percentage desc;


with q as(
    select user_type,
    ntile(4) over(order by start_time) as quarter
    from bluebikes_2019
    )
select user_type, quarter,
count(*) as rides,
count(*) / (cast(sum(count(*)) over(partition by quarter) as float)) as percentage
from q
group by quarter, user_type
order by quarter, percentage desc;

(8 row(s) affected)

(8 row(s) affected)

Total execution time: 00:00:03.576

user_type,quarter,rides,percentage
Subscriber,1,386792,0.8751900658895083
Customer,1,55160,0.12480993411049164
Subscriber,2,339276,0.7676761277242777
Customer,2,102676,0.23232387227572224
Subscriber,3,333140,0.7537939726349754
Customer,3,108811,0.24620602736502462
Subscriber,4,377469,0.8540969473991461
Customer,4,64482,0.14590305260085393


user_type,quarter,rides,percentage
Subscriber,1,520314,0.8250636263448746
Customer,1,110321,0.17493637365512538
Subscriber,2,465746,0.7385361398211958
Customer,2,164888,0.2614638601788042
Subscriber,3,479890,0.7609643628475471
Customer,3,150744,0.23903563715245293
Subscriber,4,522517,0.8285582445602362
Customer,4,108117,0.17144175543976378


In [182]:
-- Rider type by gender by month

select
case when user_gender = 1 then 'male' else 'female' end as gender,
To_Char(start_time, 'Month') as month,
count(*) as rides,
round(count(*) / sum(count(*)) over(partition by To_Char(start_time, 'Month')), 4) * 100 as percentage
from bluebikes_2018
group by extract(month from start_time), month, gender
order by extract(month from start_time), rides desc

(24 row(s) affected)

Total execution time: 00:00:02.471

gender,month,rides,percentage
male,January,30689,74.9800
female,January,10243,25.0200
male,February,45380,72.2400
female,February,17437,27.7600
male,March,44814,71.1500
female,March,18171,28.8500
male,April,65357,66.5600
female,April,32837,33.4400
male,May,113058,63.2100
female,May,65807,36.7900


In [22]:
-- Rider type by gender by YEAR and month

select
case when user_gender = 1 then 'male' else 'female' end as gender,
To_Char(start_time, 'YYYY') as year,
To_Char(start_time, 'Month') as month,
count(*) as rides,
round(count(*) / sum(count(*)) over(partition by To_Char(start_time, 'Month')), 4) * 100 as percentage
from bluebikes_combined
group by extract(year from start_time), extract(month from start_time), year, month, gender
order by extract(month from start_time), extract(year from start_time), rides desc

(48 row(s) affected)

Total execution time: 00:00:10.527

gender,year,month,rides,percentage
male,2018,January,30689,27.7000
female,2018,January,10243,9.2400
male,2019,January,52134,47.0500
female,2019,January,17738,16.0100
male,2018,February,45380,31.6700
female,2018,February,17437,12.1700
male,2019,February,59541,41.5500
female,2019,February,20925,14.6000
male,2018,March,44814,27.1000
female,2018,March,18171,10.9900


In [183]:
-- Riders by age group

select 
count(1) as riders,
-- 2018 - trunc(cast(user_birth_year as numeric), 0), # this is for sanity testing
case 
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '12 years' then 'under_12'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '20 years' then 'teenager'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '30 years' then '20s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '40 years' then '30s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '50 years' then '40s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '60 years' then '50s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '70 years' then '60s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '80 years' then '70s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '90 years' then '80s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '90 years' then '80s'
    else 'older' end as age_group,
round(count(1) / sum(count(1)) over(), 4) * 100 as percentage
from bluebikes_2018
group by age_group
order by percentage desc
-- limit 500

(9 row(s) affected)

Total execution time: 00:00:04.440

riders,age_group,percentage
699781,20s,39.5800
465235,30s,26.3200
383585,40s,21.7000
122924,50s,6.9500
48958,60s,2.7700
30925,teenager,1.7500
9994,older,0.5700
6125,70s,0.3500
279,80s,0.0200


In [184]:
-- Riders by gender and age group

select count(1) as riders,
       case
           when user_gender = 1 then 'male'
           when user_gender = 2 then 'female'
           else 'unspecified'
       end as gender, -- 2018 - trunc(cast(user_birth_year as numeric), 0), # this is for sanity testing
case
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '12 years' then 'under_12'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '20 years' then 'teenager'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '30 years' then '20s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '40 years' then '30s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '50 years' then '40s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '60 years' then '50s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '70 years' then '60s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '80 years' then '70s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '90 years' then '80s'
    when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '90 years' then '80s'
    else 'older'
    end as age_group,
round(count(1) / sum(count(1)) over(), 4) * 100 as percentage
from bluebikes_2018
group by age_group, gender
order by age_group, gender -- limit 500

(26 row(s) affected)

Total execution time: 00:00:04.580

riders,gender,age_group,percentage
198274,female,20s,11.2200
497868,male,20s,28.1600
3639,unspecified,20s,0.2100
111795,female,30s,6.3200
349456,male,30s,19.7700
3984,unspecified,30s,0.2300
35141,female,40s,1.9900
139249,male,40s,7.8800
209195,unspecified,40s,11.8300
32777,female,50s,1.8500


In [185]:
-- Riders by gender and age group WORKING!!

select riders,
       gender,
       age_group,
       total_percent,
       round(riders / sum(riders) over(partition by age_group), 4) * 100 as age_group_percent
from
    (select count(1) as riders,
    case
        when user_gender = 1 then 'male'
        when user_gender = 2 then 'female'
        else 'unspecified'
    end as gender, 
    -- 2018 - trunc(cast(user_birth_year as numeric), 0), # this is for sanity testing
    case
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '12 years' then 'under_12'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '20 years' then 'teenager'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '30 years' then '20s'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '40 years' then '30s'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '50 years' then '40s'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '60 years' then '50s'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '70 years' then '60s'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '80 years' then '70s'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '90 years' then '80s'
        when (trunc(cast(user_birth_year as numeric), 0) || '-01-01')::timestamp > '2018-01-01'::timestamp - interval '90 years' then '80s'
        else 'older'
    end as age_group,
    round(count(1) / sum(count(1)) over(), 4) * 100 as total_percent
    from bluebikes_2018
    group by age_group, gender
        order by age_group, gender
        ) big

group by age_group, gender, riders, total_percent
order by age_group, gender

(26 row(s) affected)

Total execution time: 00:00:04.934

riders,gender,age_group,total_percent,age_group_percent
198274,female,20s,11.2200,28.3300
497868,male,20s,28.1600,71.1500
3639,unspecified,20s,0.2100,0.5200
111795,female,30s,6.3200,24.0300
349456,male,30s,19.7700,75.1100
3984,unspecified,30s,0.2300,0.8600
35141,female,40s,1.9900,9.1600
139249,male,40s,7.8800,36.3000
209195,unspecified,40s,11.8300,54.5400
32777,female,50s,1.8500,26.6600


In [186]:
-- Percentage by gender

select
case when user_gender = 1 then 'male' 
    when user_gender = 2 then 'female'
    else 'unspecified' end as gender,
count(1),
round(count(1) / sum(count(1)) over(), 6) * 100 as percentage
from bluebikes_2018
group by user_gender

(3 row(s) affected)

Total execution time: 00:00:00.993

gender,count,percentage
unspecified,227677,12.879100
male,1141735,64.584900
female,398394,22.536100


In [35]:
-- Days of the week percent change 2018 2019
with 
dow2018 as(
    select count(1) as riders,
    To_Char(start_time, 'Day') as weekday
    from bluebikes_2018
    group by weekday
    order by riders desc
),

dow2019 as(
    select count(1) as riders,
    To_Char(start_time, 'Day') as weekday
    from bluebikes_2019
    group by weekday
    order by riders desc
)

select 
dow2018.weekday,
dow2018.riders as riders2018,
dow2019.riders as riders2019,
dow2019.riders - dow2018.riders as riders_change,
round((dow2019.riders - dow2018.riders)::NUMERIC / dow2018.riders, 4) * 100 || ' %' as riders_percent_change

from dow2018
join dow2019 on dow2018.weekday = dow2019.weekday


(7 row(s) affected)

Total execution time: 00:00:01.668

weekday,riders2018,riders2019,riders_change,riders_percent_change
Thursday,282700,386579,103879,36.7500 %
Sunday,191055,285203,94148,49.2800 %
Tuesday,265161,384455,119294,44.9900 %
Wednesday,288485,389421,100936,34.9900 %
Monday,256032,372387,116355,45.4500 %
Saturday,208144,319024,110880,53.2700 %
Friday,276229,385468,109239,39.5500 %


In [188]:
-- Riders by month

select count(1) as riders,
To_Char(start_time, 'Month') as month
from bluebikes_2018
-- where date_trunc('day', start_time) = date_trunc('day', timestamp '2018-10-01')
group by month
order by riders desc

(12 row(s) affected)

Total execution time: 00:00:01.445

riders,month
242916,July
236182,September
236076,August
205359,June
200100,October
178865,May
121419,November
98194,April
81961,December
62985,March


In [189]:
-- Top and Bottom 5 Stations based on riders

with top5 as
    (select number, name, district,
    count(1) as riders,
    'top5' as placement
    from bluebikes_stations
    join bluebikes_2018 on bluebikes_stations.id = bluebikes_2018.start_station_id
    group by number, name, district, placement
    order by riders desc
    limit 5),

bottom5 as
    (select number, name, district,
    count(1) as riders,
    'bottom5' as placement
    from bluebikes_stations
    join bluebikes_2018 on bluebikes_stations.id = bluebikes_2018.start_station_id
    group by number, name, district, placement
    order by riders 
    limit 5)

select * from top5
union all
select * from bottom5
order by riders desc

(10 row(s) affected)

Total execution time: 00:00:02.134

number,name,district,riders,placement
M32006,MIT at Mass Ave / Amherst St,Cambridge,53846,top5
M32005,MIT Stata Center at Vassar St / Main St,Cambridge,41559,top5
M32011,Central Square at Mass Ave / Essex St,Cambridge,37346,top5
A32010,South Station - 700 Atlantic Ave,Boston,36346,top5
M32041,MIT Pacific St at Purrington St,Cambridge,29379,top5
C32049,Thetford Ave at Norfolk St,Boston,15,bottom5
C32044,Blue Hill Ave at Almont St,Boston,12,bottom5
C32050,Mattapan Library,Boston,12,bottom5
C32059,Belgrade Ave at Walworth St,Boston,9,bottom5
C23045,Morton St T,Boston,5,bottom5


In [45]:
-- Sort by Duration
with
dur2018 as(
    select 
    count(1) as riders,
    case 
        when date_trunc('second', end_time - start_time) < time '00:05:00' then 'under5'
        when date_trunc('second', end_time - start_time) < time '00:10:00' and date_trunc('second', end_time - start_time) > '00:05:00' then '5to10'
        when date_trunc('second', end_time - start_time) < time '00:15:00' and date_trunc('second', end_time - start_time) > '00:10:00' then '10to15'
        when date_trunc('second', end_time - start_time) < time '00:20:00' and date_trunc('second', end_time - start_time) > '00:15:00' then '15to20'
        when date_trunc('second', end_time - start_time) < time '00:25:00' and date_trunc('second', end_time - start_time) > '00:20:00' then '20to25'
        when date_trunc('second', end_time - start_time) < time '00:30:00' and date_trunc('second', end_time - start_time) > '00:25:00' then '25to30'
        when date_trunc('second', end_time - start_time) < time '00:35:00' and date_trunc('second', end_time - start_time) > '00:30:00' then '30to35'
        when date_trunc('second', end_time - start_time) < time '00:40:00' and date_trunc('second', end_time - start_time) > '00:35:00' then '35to40'
        when date_trunc('second', end_time - start_time) < time '00:45:00' and date_trunc('second', end_time - start_time) > '00:40:00' then '40to45'
        when date_trunc('second', end_time - start_time) < time '00:50:00' and date_trunc('second', end_time - start_time) > '00:45:00' then '45to50'
        when date_trunc('second', end_time - start_time) < time '00:55:00' and date_trunc('second', end_time - start_time) > '00:50:00' then '50to55'
        when date_trunc('second', end_time - start_time) < time '01:00:00' and date_trunc('second', end_time - start_time) > '00:55:00' then '55to60'
        when date_trunc('second', end_time - start_time) < time '02:00:00' and date_trunc('second', end_time - start_time) > '01:00:00' then '1hrto2hr'
        when date_trunc('second', end_time - start_time) < time '03:00:00' and date_trunc('second', end_time - start_time) > '02:00:00' then '2hrto3hr'
        when date_trunc('second', end_time - start_time) < time '04:00:00' and date_trunc('second', end_time - start_time) > '03:00:00' then '3hrto4hr'
        when date_trunc('second', end_time - start_time) < time '05:00:00' and date_trunc('second', end_time - start_time) > '04:00:00' then '4hrto5hr'
        when date_trunc('second', end_time - start_time) < time '06:00:00' and date_trunc('second', end_time - start_time) > '05:00:00' then '5hrto6hr'
        when date_trunc('second', end_time - start_time) < time '12:00:00' and date_trunc('second', end_time - start_time) > '06:00:00' then '6hrto12hr'
        when date_trunc('second', end_time - start_time) < time '24:00:00' and date_trunc('second', end_time - start_time) > '12:00:00' then '12hrto24hr'
    else 'longer' end as duration

    from bluebikes_2018
    group by duration
    order by riders desc
    ),

dur2019 as (
    select 
    count(1) as riders,
    case 
        when date_trunc('second', end_time - start_time) < time '00:05:00' then 'under5'
        when date_trunc('second', end_time - start_time) < time '00:10:00' and date_trunc('second', end_time - start_time) > '00:05:00' then '5to10'
        when date_trunc('second', end_time - start_time) < time '00:15:00' and date_trunc('second', end_time - start_time) > '00:10:00' then '10to15'
        when date_trunc('second', end_time - start_time) < time '00:20:00' and date_trunc('second', end_time - start_time) > '00:15:00' then '15to20'
        when date_trunc('second', end_time - start_time) < time '00:25:00' and date_trunc('second', end_time - start_time) > '00:20:00' then '20to25'
        when date_trunc('second', end_time - start_time) < time '00:30:00' and date_trunc('second', end_time - start_time) > '00:25:00' then '25to30'
        when date_trunc('second', end_time - start_time) < time '00:35:00' and date_trunc('second', end_time - start_time) > '00:30:00' then '30to35'
        when date_trunc('second', end_time - start_time) < time '00:40:00' and date_trunc('second', end_time - start_time) > '00:35:00' then '35to40'
        when date_trunc('second', end_time - start_time) < time '00:45:00' and date_trunc('second', end_time - start_time) > '00:40:00' then '40to45'
        when date_trunc('second', end_time - start_time) < time '00:50:00' and date_trunc('second', end_time - start_time) > '00:45:00' then '45to50'
        when date_trunc('second', end_time - start_time) < time '00:55:00' and date_trunc('second', end_time - start_time) > '00:50:00' then '50to55'
        when date_trunc('second', end_time - start_time) < time '01:00:00' and date_trunc('second', end_time - start_time) > '00:55:00' then '55to60'
        when date_trunc('second', end_time - start_time) < time '02:00:00' and date_trunc('second', end_time - start_time) > '01:00:00' then '1hrto2hr'
        when date_trunc('second', end_time - start_time) < time '03:00:00' and date_trunc('second', end_time - start_time) > '02:00:00' then '2hrto3hr'
        when date_trunc('second', end_time - start_time) < time '04:00:00' and date_trunc('second', end_time - start_time) > '03:00:00' then '3hrto4hr'
        when date_trunc('second', end_time - start_time) < time '05:00:00' and date_trunc('second', end_time - start_time) > '04:00:00' then '4hrto5hr'
        when date_trunc('second', end_time - start_time) < time '06:00:00' and date_trunc('second', end_time - start_time) > '05:00:00' then '5hrto6hr'
        when date_trunc('second', end_time - start_time) < time '12:00:00' and date_trunc('second', end_time - start_time) > '06:00:00' then '6hrto12hr'
        when date_trunc('second', end_time - start_time) < time '24:00:00' and date_trunc('second', end_time - start_time) > '12:00:00' then '12hrto24hr'
    else 'longer' end as duration

    from bluebikes_2019
    group by duration
    order by riders desc
)

select dur2018.duration, 
dur2018.riders as riders2018, 
dur2019.riders as riders2019,
dur2019.riders - dur2018.riders as riders_change,
round((dur2019.riders - dur2018.riders)::NUMERIC / dur2018.riders, 4) * 100 || ' %' as riders_percent_change,
round(((dur2019.riders - dur2018.riders)::numeric / dur2018.riders) - ((sum(dur2019.riders) over() - sum(dur2018.riders) over()) / sum(dur2018.riders) over()), 4) * 100 || ' %' as pct_change_relative
from dur2018
join dur2019 on dur2018.duration = dur2019.duration
order by 

(20 row(s) affected)

Total execution time: 00:00:04.619

duration,riders2018,riders2019,riders_change,riders_percent_change,pct_change_relative
10to15,356463,516544,160081,44.9100 %,2.2200 %
20to25,142316,206478,64162,45.0800 %,2.3900 %
25to30,88795,130301,41506,46.7400 %,4.0500 %
6hrto12hr,1244,1405,161,12.9400 %,-29.7500 %
1hrto2hr,45577,58847,13270,29.1200 %,-13.5800 %
5to10,524937,730680,205743,39.1900 %,-3.5000 %
5hrto6hr,481,503,22,4.5700 %,-38.1200 %
12hrto24hr,1115,1396,281,25.2000 %,-17.4900 %
3hrto4hr,2304,2128,-176,-7.6400 %,-50.3300 %
2hrto3hr,7685,7623,-62,-0.8100 %,-43.5000 %


In [198]:
-- Top 3 Positive and Negative Bike Check-in difference per month

with outbikes as (
    select
    stations.name,
    extract(month from start_time) as month_no,
    To_Char(start_time, 'Month') as month,
    count(start_station_id) as outgoing
    from bluebikes_2018 bikes
    join bluebikes_stations stations on stations.id = bikes.start_station_id
    group by month_no, month, stations.name
    order by month_no, month, outgoing desc
),

inbikes as(
    select
    stations.name,
    extract(month from start_time) as month_no,
    To_Char(start_time, 'Month') as month,
    count(end_station_id) as incoming
    from bluebikes_2018 bikes
    join bluebikes_stations stations on stations.id = bikes.end_station_id
    group by month_no, month, stations.name
    order by month_no, month, incoming desc
),

positivebikeflow as (
    select 
    row_number() over(partition by outbikes.month order by outbikes.month, sum(inbikes.incoming) - sum(outbikes.outgoing) desc) as rank,
    outbikes.month_no as month_no,
    outbikes.month as month, 
    outbikes.name as name, 
    sum(outbikes.outgoing) as outgoing,
    sum(inbikes.incoming) as incoming,
    sum(inbikes.incoming) - sum(outbikes.outgoing) as difference
    from outbikes
    join inbikes on outbikes.name = inbikes.name
    group by outbikes.month_no, outbikes.month, outbikes.name
    order by outbikes.month_no, outbikes.month, difference desc
),

negativebikeflow as (
    select 
    row_number() over(partition by outbikes.month order by outbikes.month, sum(inbikes.incoming) - sum(outbikes.outgoing)) as rank,
    outbikes.month_no as month_no,
    outbikes.month as month,
    outbikes.name as name,
    sum(outbikes.outgoing) as outgoing,
    sum(inbikes.incoming) as incoming,
    sum(inbikes.incoming) - sum(outbikes.outgoing) as difference
    from outbikes
    join inbikes on outbikes.name = inbikes.name
    group by outbikes.month_no, outbikes.month, outbikes.name
    order by outbikes.month_no, outbikes.month, difference
)

select * from positivebikeflow
where rank in (1,2,3)
union all
select * from negativebikeflow
where rank in (1,2,3)
order by month_no, difference desc

(72 row(s) affected)

Total execution time: 00:00:04.732

rank,month_no,month,name,outgoing,incoming,difference
1,1.0,January,MIT Stata Center at Vassar St / Main St,13968,46369,32401
2,1.0,January,MIT at Mass Ave / Amherst St,18984,49635,30651
3,1.0,January,Nashua Street at Red Auerbach Way,10104,37861,27757
3,1.0,January,Congress St at Boston City Hall,716,1040,324
2,1.0,January,Roxbury YMCA - Warren St at MLK Blvd,456,759,303
1,1.0,January,Grove Hall Library - 41 Geneva Ave,180,336,156
1,2.0,February,MIT Stata Center at Vassar St / Main St,21048,46369,25321
2,2.0,February,Nashua Street at Red Auerbach Way,14196,37861,23665
3,2.0,February,MIT at Mass Ave / Amherst St,26832,49635,22803
3,2.0,February,NCAAA - Walnut Ave at Crawford St,348,500,152


(1 row(s) affected)

Total execution time: 00:00:01.875

date_part
1.0
